In [1]:
import pandas as pd

covidData = pd.read_csv('../data/interim/covidData.csv', index_col=0)
rel_census_data = pd.read_csv('../data/interim/rel_census_data.csv', index_col=0)
rent_clean = pd.read_csv('../data/interim/rent_clean.csv', index_col=0)
inv_clean = pd.read_csv('../data/interim/inv_clean.csv', index_col=0)

## Final Dataset Creation

#### Combining rent and inventory data

Melt rent and inventory data to join in the right format. Create relevant time features. 

In [2]:
from dateutil.relativedelta import relativedelta

combined_data = rent_clean
combined_data = combined_data.melt(id_vars=['zipcode','bd'])

#Create new time features
combined_data['timestamp'] = pd.to_datetime(combined_data.variable,format='%Y-%m')
combined_data['month'] = combined_data['timestamp'].dt.month
combined_data['quarter'] = combined_data['timestamp'].dt.quarter
combined_data['year'] = combined_data['timestamp'].dt.year

combined_data['prev_timestamp'] = combined_data['timestamp'].apply(lambda x: x - relativedelta(months=1))

In [3]:
# Create previous month rent price feature

for i in range(len(combined_data)):
    row_zipcode = combined_data.zipcode.iloc[i]
    row_bd = combined_data.bd.iloc[i]
    row_prev_timestamp = combined_data.prev_timestamp.iloc[i]

    try:
        prev_rent_price = combined_data['value'][(combined_data.zipcode == row_zipcode) & (combined_data.timestamp == row_prev_timestamp) & (combined_data.bd == row_bd)].values
        combined_data.loc[i, 'prev_month_rent'] = prev_rent_price
    except Exception as e: 
        continue

In [4]:
from dateutil.relativedelta import relativedelta

combine_inv = inv_clean
combine_inv = combine_inv.melt(id_vars=['zipcode','bd'])

#Create new time features
combine_inv['timestamp'] = pd.to_datetime(combine_inv.variable,format='%Y-%m')
combine_inv['month'] = combine_inv['timestamp'].dt.month
combine_inv['quarter'] = combine_inv['timestamp'].dt.quarter
combine_inv['year'] = combine_inv['timestamp'].dt.year

combine_inv['prev_timestamp'] = combine_inv['timestamp'].apply(lambda x: x - relativedelta(months=1))

for i in range(len(combine_inv)):
    row_zipcode = combine_inv.zipcode.iloc[i]
    row_bd = combine_inv.bd.iloc[i]
    row_prev_timestamp = combine_inv.prev_timestamp.iloc[i]

    try:
        prev_inventory = combine_inv['value'][(combine_inv.zipcode == row_zipcode) & (combine_inv.timestamp == row_prev_timestamp) & (combine_inv.bd == row_bd)].values
        combine_inv.loc[i, 'prev_inventory'] = prev_inventory
    except Exception as e: 
        continue

combine_inv.insert(0, 'inventory', combine_inv['value'])
combine_inv.drop(['variable','value','timestamp','prev_timestamp'],axis=1,inplace=True)
combine_inv.dropna(inplace=True)

In [5]:
# merge rent and inventory data
combined_data=combined_data.merge(combine_inv,how='inner',on=['zipcode','bd','month','quarter','year'])

cpy = combined_data
cpy.to_csv('../data/interim/combined_data_interim.csv')
combined_data.head(3)

,zipcode,bd,variable,value,timestamp,month,quarter,year,prev_timestamp,prev_month_rent,inventory,prev_inventory
0,10001,0.0,2018-07,2850.0,2018-07-01,7,3,2018,2018-06-01,2834.0,250.0,262.0
1,10001,1.0,2018-07,3848.0,2018-07-01,7,3,2018,2018-06-01,3872.0,424.0,409.0
2,10001,2.0,2018-07,5595.0,2018-07-01,7,3,2018,2018-06-01,5800.0,247.0,245.0


In [6]:
covid_data_melt = covidData.melt(id_vars='zipcode')

# Parse timestamp and type of data
covid_data_melt['type'] = covid_data_melt['variable'].str.split('-',expand=True)[1]
covid_data_melt['timestamp'] = pd.to_datetime('2020-' + covid_data_melt['variable'].str[:3],format="%Y-%b")

# Joining Covid Data

# % Positive for current month and previous month
percent_positive = covid_data_melt[covid_data_melt['type'] == 'Percent Positive'][['zipcode','timestamp','value']]
percent_positive.columns = ['zipcode','timestamp','covid_%_positive']

combined_data = pd.merge(combined_data, percent_positive,  how='left', on=['timestamp','zipcode'])
combined_data = pd.merge(combined_data, percent_positive,  how='left', left_on=['prev_timestamp','zipcode'], right_on=['timestamp','zipcode'], suffixes=('','_prev_mo'))

# Rate of Increase for current month and previous month

rate_of_increase = covid_data_melt[covid_data_melt['type'] == 'Rate of Increase'][['zipcode','timestamp','value']]
rate_of_increase.columns = ['zipcode','timestamp','covid_rate_of_increase']

combined_data = pd.merge(combined_data, rate_of_increase,  how='left', on=['timestamp','zipcode'])
combined_data = pd.merge(combined_data, rate_of_increase,  how='left', left_on=['prev_timestamp','zipcode'], right_on=['timestamp','zipcode'], suffixes=('','_prev_mo'))

combined_data = combined_data.drop(['timestamp_prev_mo'],axis=1)

# Fill NA in covid cols
covid_cols = ['covid_%_positive','covid_%_positive_prev_mo','covid_rate_of_increase','covid_rate_of_increase_prev_mo']
combined_data[covid_cols] = combined_data[covid_cols].fillna(value=0)

In [7]:
# Renaming and moving rent column
combined_data.insert(0, 'rent', combined_data['value'])

# Removing columns that won't be used as features now and dropping earliest month values (no previous month rent feature for those)
combined_data.drop(['variable','value','timestamp','prev_timestamp'],axis=1,inplace=True)
combined_data.dropna(inplace=True)

# Encoding dummy variables for categorical columns

# Month
month_dummies = pd.get_dummies(combined_data['month'],prefix='month')
combined_data = pd.concat([combined_data,month_dummies],axis=1)

# Quarter
# quarter_dummies = pd.get_dummies(combined_data['quarter'],prefix='quarter')
# combined_data = pd.concat([combined_data,quarter_dummies],axis=1)

# Year

year_dummies = pd.get_dummies(combined_data['year'],prefix='year')
combined_data = pd.concat([combined_data,year_dummies],axis=1)

# Bedrooms
bd_dummies = pd.get_dummies(combined_data['bd'],prefix='bd_')
combined_data = pd.concat([combined_data,bd_dummies],axis=1)

# Drop categorical columns
combined_data.drop(['month','quarter','year','bd'],axis=1,inplace=True)

In [8]:
# Add Census Data
combined_data = pd.merge(combined_data,rel_census_data,left_on='zipcode',right_on='zip_code_tabulation_area_(5-digit)').drop('zip_code_tabulation_area_(5-digit)',axis=1)


# Drop final cols (used to join)
combined_data.drop(['zipcode'],axis=1,inplace=True)
combined_data.drop(['inventory'],axis=1,inplace=True)

In [9]:
combined_data.to_csv('../data/processed/combined_data.csv')